## Globals

In [16]:
from __future__ import print_function

import sys
import os
import re
import glob
import subprocess
import shutil
import json

#root_dir = '/home/preclineu/yartoe'
root_dir = '/project_cephfs/3022017.06/OASIS2/'
cluster_log_dir = os.path.join(root_dir,'logs')
#aroma_path = '/home/preclineu/andmar/sfw/ICA-AROMA/ICA_AROMA.py'  

cmd_qsub_base = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '32gb',
                 '-logfiledir', cluster_log_dir
                ]

#abcd_dict = os.path.join(root_dir,'downloads','abcd_modalities.json')

# read the dictionary
#with open(abcd_dict,'r') as f:
   #modalities = json.load(f) 
    
# get a list of subjects to process
sub_dirs = sorted(glob.glob(os.path.join(root_dir,'freesurfer','*_MR*')))

#sub_dirs = sub_dirs[0:15]
#print('\n'.join(sub_dirs))

#print('Dictionary read from:', abcd_dict)
print('Found', len(sub_dirs), 'subjects to process')
sub_dirs

Found 373 subjects to process


['/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0001_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0001_MR2',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0002_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0002_MR2',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0002_MR3',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0004_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0004_MR2',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0005_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0005_MR2',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0005_MR3',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0007_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0007_MR3',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0007_MR4',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0008_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0008_MR2',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_000

## run fsl_anat

In [17]:
verbose = False

sub_batch = sub_dirs[:10]#[10000:]

submitted = 0
already_processed = 0
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0]
    if verbose:
        print('processing subject',subid)

    job_name = 'ABCD_fsl_anat_' + subid
    
    sessions = glob.glob(os.path.join(s, 'ses-*'))
    for sess in sessions:
        sdir = os.path.join(sess, 'anat')
        if not os.path.exists(sdir):
            continue
        
        # get all subdirectories 
        t1_all = [os.path.join(sdir, o) for o in os.listdir(sdir) if os.path.isdir(os.path.join(sdir, o))]

        for t1_dir in t1_all:
            if os.path.exists(os.path.join(t1_dir,'T1.anat','T1_biascorr_brain.nii.gz')):
                if verbose: 
                    print(' >', subid, ' anatomical data already processed. Doing nothing')
                already_processed += 1
                continue
    
            cmd_fsl_anat = ['fsl_anat -i ', os.path.join(t1_dir,'T1.nii.gz')]
            cmd_str = '"%s"' % str(' '.join(cmd_fsl_anat)) 
            cmd_qsub = cmd_qsub_base + ['-name', job_name,'-command', cmd_str]

            #subprocess.Popen(' '.join(cmd_qsub), shell=True)
            
            submitted += 1
    
print('number of jobs submitted =', submitted)
print('number of T1s already processed =', already_processed)

number of jobs submitted = 4
number of T1s already processed = 24


## Select subjects (e.g. only those having 4 year follow-up)

In [36]:
target_ses = '_MR*'
#target_ses = '_MRI2'
#target_ses = '_MRI3'

sub_list = []
sub_dirs_target = []
sub_list_short = []

#for s in sub_dirs:
#    subid = os.path.splitext(os.path.basename(s))[0] 
#    ses_dirs = glob.glob(os.path.join(s))
    #print(ses_dirs)
#    ses_dirs_short = os.path.join(('/project_' + s.split("_")[1]))
#    sub_list_short.append(ses_dirs_short)
    
#    sessions = list(map(os.path.basename, ses_dirs))
#    if target_ses in sessions:
#        sub_list.append(subid)
#        sub_dirs_target.append(s)

#print(len(sub_list), 'subjects found')


for s in sub_dirs:
    subid = os.path.splitext(os.path.basename(s))[0]
    subid_path = os.path.dirname(s) +"/" + subid.split("_")[0] + "_" + subid.split("_")[1]
    subid_short = subid.split("_")[0] + "_" + subid.split("_")[1]
    sub_list_short.append(subid_short)
    sub_list.append(subid_path)

sub_list

['/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0001',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0001',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0002',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0002',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0002',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0004',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0004',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0005',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0005',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0005',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0007',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0007',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0007',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0008',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0008',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0009',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0009

In [43]:
# change the subject list for OASIS
#sub_list[:10]
#os.path.join('/project', s.split("_")[1])
single_subs = set(sub_list_short)
single_subs = list(single_subs)
single_subs

sub_batch = set(sub_list)
sub_batch = list(sub_batch)
sub_batch


['/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0027',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0108',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0022',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0172',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0008',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0028',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0140',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0146',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0023',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0067',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0060',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0048',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0141',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0035',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0157',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0078',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0096

## Run Freesurfer (cross-sectional)

In [ ]:
num_processed = 0
num_failed = 0
num_submitted = 0
sub_no_t1 = []

verbose = False

default_t1_type = 'ABCD-T1-NORM'
log_prefix = 'ABCD_fs_'

#sub_batch = sub_dirs[100:]
sub_batch = sub_dirs_target

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_batch), 'subjects ...')
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0] 
    
    sessions = glob.glob(os.path.join(s, 'ses-*'))
    for sess in sessions:
        sdir = os.path.join(sess, 'anat')
        sesid = os.path.basename(sess)
        full_subid = '_'.join((subid, sesid))
        
        if not os.path.exists(sdir):
            continue
        
        # get all subdirectories 
        t1_all = [os.path.join(sdir, o) for o in os.listdir(sdir) if os.path.isdir(os.path.join(sdir, o))]
        
        if len(t1_all) > 1:
            if verbose:
                print(' > Multiple t1s found for', full_subid, '(',len(t1_all), ') using default')
            
            # sub-select only ABCD-T1-NORM
            t1_all = [k for k in t1_all if default_t1_type in k]
            if len(t1_all) > 1:
                if verbose:
                    print(' > Failed.')
                continue;                        
        
        for t1_dir in t1_all:
            t1_file = os.path.join(t1_dir,'T1.nii.gz')            
            out_dir = os.path.join(root_dir, 'freesurfer', full_subid)
            
            if os.path.exists(out_dir):
                if os.path.exists(os.path.join(out_dir, 'stats', 'rh.w-g.pct.stats')):
                    # analysis is complete
                    if verbose:
                        print(' > Freesurfer already run for', full_subid,'doing nothing')
                    num_processed += 1
                    continue
                else:
                    # analysis did not complete
                    num_failed +=1
                    if verbose:
                        print(' > Freesurfer subject',full_subid,'found but failed. Continuing analysis')
                    cmd_fs = ['recon-all -s', full_subid, '-all','-qcache']
            else:
                # no analysis has been run
                if not os.path.exists(t1_file):
                    if verbose: 
                        print(' > No T1 found for sub', full_subid, '. Doing nothing')
                    sub_no_t1.append(subid)
                    continue
        
                cmd_fs = ['recon-all -i ', t1_file, '-s', full_subid, '-all','-qcache']
            
            #print(' '.join(cmd_fs))

            cmd_env = [fs_env + ' ; ',
                       'export SUBJECTS_DIR=' + os.path.join(root_dir,'freesurfer ; ')]
            cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
            cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(full_subid),'-command', cmd_str]
    
            num_submitted += 1
            print(' '.join(cmd_qsub))
            #subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)
print('number without t1 =',len(sub_no_t1))

## Create Freesurfer longitudinal templates

Note: this needs to be run more carefully than the cross sectional approach above because it just clobbers the template directory. 

In [77]:
sessions

for i in range(len(sessions)):
    if "BL" in sessions[i]:
        bl = sessions[i]

subid + sesid

'000049148426000049148426_FU3'

In [46]:
num_processed = 0
num_failed = 0
num_submitted = 0

verbose = False

log_prefix = 'OASIS2_longsf_'

#sub_batch = sub_dirs[10:]
#sub_batch = sub_dirs_target[5900:]
sub_batch = sub_batch

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_batch), 'subjects ...')
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0] 
    
    cmd_fs = ['recon-all -base ', subid + '_long',]
    
    sessions = sorted(glob.glob(os.path.join(s + '_*')))
    
    # make sure the baseline session is first 
    for i in range(len(sessions)):
        if 'MR1' in sessions[i]:
            bl = sessions[i]
            del sessions[i]
            sessions.insert(0, bl)
            continue
    
    good_sessions = 0 
    for sess in sessions:
        sesid = os.path.basename(sess)
        #full_subid = '_'.join((subid, sesid)) 
        full_subid = sesid
        fs_sess_dir = os.path.join(root_dir, 'freesurfer', full_subid)
        
        fs_sessions = []
        if os.path.exists(os.path.join(fs_sess_dir, 'stats', 'rh.w-g.pct.stats')):
            fs_sessions.append(full_subid)
        else:
            print(' > Freesurfer did not complete for', full_subid)
            continue
        
        cmd_fs += ['-tp', full_subid]
        good_sessions += 1
        
    if good_sessions < 2:
        print(' > Only one timepoint for', full_subid, 'aborting.')
        continue
        
    cmd_fs += ['-all', '-qcache']
        
    print(' '.join(cmd_fs))

    cmd_env = [fs_env + ' ; ',
               'export SUBJECTS_DIR=' + os.path.join(root_dir,'freesurfer ; ')]
    cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
    cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(subid + '_long'),'-command', cmd_str]
    num_submitted += 1
    #print(' '.join(cmd_qsub))
    
    subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)

Processing 150 subjects ...
recon-all -base  OAS2_0027_long -tp OAS2_0027_MR1 -tp OAS2_0027_MR2 -tp OAS2_0027_MR3 -tp OAS2_0027_MR4 -all -qcache
recon-all -base  OAS2_0108_long -tp OAS2_0108_MR1 -tp OAS2_0108_MR2 -all -qcache
recon-all -base  OAS2_0022_long -tp OAS2_0022_MR1 -tp OAS2_0022_MR2 -all -qcache
recon-all -base  OAS2_0172_long -tp OAS2_0172_MR1 -tp OAS2_0172_MR2 -all -qcache
recon-all -base  OAS2_0008_long -tp OAS2_0008_MR1 -tp OAS2_0008_MR2 -all -qcache
recon-all -base  OAS2_0028_long -tp OAS2_0028_MR1 -tp OAS2_0028_MR2 -all -qcache
recon-all -base  OAS2_0140_long -tp OAS2_0140_MR1 -tp OAS2_0140_MR2 -tp OAS2_0140_MR3 -all -qcache
recon-all -base  OAS2_0146_long -tp OAS2_0146_MR1 -tp OAS2_0146_MR2 -all -qcache
recon-all -base  OAS2_0023_long -tp OAS2_0023_MR1 -tp OAS2_0023_MR2 -all -qcache
recon-all -base  OAS2_0067_long -tp OAS2_0067_MR1 -tp OAS2_0067_MR2 -tp OAS2_0067_MR3 -tp OAS2_0067_MR4 -all -qcache
recon-all -base  OAS2_0060_long -tp OAS2_0060_MR1 -tp OAS2_0060_MR2 -all

recon-all -base  OAS2_0044_long -tp OAS2_0044_MR1 -tp OAS2_0044_MR2 -tp OAS2_0044_MR3 -all -qcache
recon-all -base  OAS2_0063_long -tp OAS2_0063_MR1 -tp OAS2_0063_MR2 -all -qcache
recon-all -base  OAS2_0058_long -tp OAS2_0058_MR1 -tp OAS2_0058_MR2 -tp OAS2_0058_MR3 -all -qcache
52765305.dccn-l029.dccn.nl
52765306.dccn-l029.dccn.nl
52765307.dccn-l029.dccn.nl
52765308.dccn-l029.dccn.nl
recon-all -base  OAS2_0131_long -tp OAS2_0131_MR1 -tp OAS2_0131_MR2 -all -qcache
52765309.dccn-l029.dccn.nl
recon-all -base  OAS2_0106_long -tp OAS2_0106_MR1 -tp OAS2_0106_MR2 -all -qcache
recon-all -base  OAS2_0159_long -tp OAS2_0159_MR1 -tp OAS2_0159_MR2 -all -qcache
52765310.dccn-l029.dccn.nl
recon-all -base  OAS2_0127_long -tp OAS2_0127_MR1 -tp OAS2_0127_MR2 -tp OAS2_0127_MR3 -tp OAS2_0127_MR4 -tp OAS2_0127_MR5 -all -qcache
52765311.dccn-l029.dccn.nl
recon-all -base  OAS2_0102_long -tp OAS2_0102_MR1 -tp OAS2_0102_MR2 -tp OAS2_0102_MR3 -all -qcache
52765312.dccn-l029.dccn.nl
recon-all -base  OAS2_0046_l

52765375.dccn-l029.dccn.nl
52765376.dccn-l029.dccn.nl
52765377.dccn-l029.dccn.nl
52765378.dccn-l029.dccn.nl
52765379.dccn-l029.dccn.nl
52765380.dccn-l029.dccn.nl
52765381.dccn-l029.dccn.nl
52765382.dccn-l029.dccn.nl
52765383.dccn-l029.dccn.nl
52765384.dccn-l029.dccn.nl
52765385.dccn-l029.dccn.nl
52765386.dccn-l029.dccn.nl
52765387.dccn-l029.dccn.nl
52765388.dccn-l029.dccn.nl
52765389.dccn-l029.dccn.nl
52765390.dccn-l029.dccn.nl
52765391.dccn-l029.dccn.nl
52765392.dccn-l029.dccn.nl
52765393.dccn-l029.dccn.nl
52765394.dccn-l029.dccn.nl
52765395.dccn-l029.dccn.nl
52765396.dccn-l029.dccn.nl
52765397.dccn-l029.dccn.nl
52765398.dccn-l029.dccn.nl
52765399.dccn-l029.dccn.nl
52765400.dccn-l029.dccn.nl
52765401.dccn-l029.dccn.nl
52765402.dccn-l029.dccn.nl
52765403.dccn-l029.dccn.nl
52765404.dccn-l029.dccn.nl


## Select subjects

In [31]:
fs_dir = os.path.join(root_dir,'freesurfer')
sub_dirs = glob.glob(os.path.join(fs_dir, '*_long'))

sub_list_target = []
for s in sub_dirs:
    subid = os.path.splitext(os.path.basename(s))[0].split('_')[0]
    
    sub_list_target.append(subid)
        
print(len(sub_list_target), 'subjects found')
sorted(sub_list_target)[:10]
sub_list_target = list(set(sub_list_target))

already_processed = glob.glob(os.path.join(fs_dir, '*.long.*'))
already_processed_list=[]
for s in already_processed:
    processed_subid = os.path.splitext(os.path.basename(s))[0].split('_')[0]
    already_processed_list.append(processed_subid)

len(list(set(already_processed)))
already_processed_list = list(set(already_processed_list))

#len(sub_list_target)
#len(already_processed_list)

leftover_list = [x for x in sub_list_target if x not in already_processed_list]
leftover_list

3561 subjects found


['000007167925',
 '000045531867',
 '000046184877',
 '000083830785',
 '000013424686',
 '000099616225',
 '000062960940',
 '000007595869',
 '000071672039',
 '000026461756',
 '000001689886',
 '000044090629',
 '000027255432',
 '000035620929',
 '000029708729',
 '000058250315',
 '000061541397',
 '000001598887',
 '000078139248',
 '000016635570',
 '000068000499',
 '000062180324',
 '000020051604',
 '000034255297',
 '000038072766',
 '000002695459',
 '000070830069',
 '000092523261',
 '000001023924',
 '000094989507',
 '000011834339',
 '000090129274',
 '000029963972',
 '000074104786',
 '000047117408',
 '000030515864',
 '000043217738',
 '000065086134',
 '000009682526',
 '000026123134',
 '000092744330',
 '000013646768',
 '000043559294',
 '000024115048',
 '000042288666',
 '000010951966',
 '000007580813',
 '000080996663',
 '000087298622',
 '000021131361',
 '000052172018',
 '000020013673',
 '000094807913',
 '000083670951',
 '000095823901',
 '000069295276',
 '000065251291',
 '000014837805',
 '000010867560

## Run longitudinal freesurfer

In [33]:
num_processed = 0
num_failed = 0
num_submitted = 0
sub_no_t1 = []

verbose = True

log_prefix = 'IMAGEN_longfs2_'

#sub_id_batch = sub_list_target
sub_id_batch = leftover_list

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_id_batch), 'subjects ...')
for subid in sub_id_batch:
        
    sessions = glob.glob(os.path.join(fs_dir, f'{subid}_*'))
    sessions = [item for item in sessions if 'long' not in item] # remove _long sessions
    
    for full_subid in sessions:
        sesid = os.path.basename(full_subid)
        
        #if not os.path.exists(os.path.join(fs_dir, full_subid)):
        #    continue
                  
        out_dir = os.path.join(root_dir, 'freesurfer', full_subid, '_longitudinal')
            
        #if os.path.exists(out_dir):
        #    if os.path.exists(os.path.join(out_dir, 'stats', 'rh.w-g.pct.stats')):
        #        # analysis is complete
        #        if verbose:
        #            print(' > Freesurfer already run for', full_subid,'doing nothing')
        #        num_processed += 1
        #        continue
        #    else:
        #         # analysis did not complete
        #        num_failed +=1
        #        if verbose:
        #            print(' > Freesurfer subject',full_subid,'found but failed. Continuing analysis')
        #        cmd_fs = ['recon-all -long ', full_subid, f'{sub_id}_long', '-all','-qcache']
        #else:
            # no analysis has been run
            #if not os.path.exists(t1_file):
            #    if verbose: 
            #        print(' > No T1 found for sub', full_subid, '. Doing nothing')
            #    sub_no_t1.append(subid)
            #    continue
        
        cmd_fs = ['recon-all -long ', full_subid, f'{subid}_long', '-all','-qcache']
            
        #print(' '.join(cmd_fs))

        cmd_env = [fs_env + ' ; ',
                    'export SUBJECTS_DIR=' + fs_dir + ' ; ']
        cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
        cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(full_subid),'-command', cmd_str]
    
        num_submitted += 1
        #print(' '.join(cmd_qsub))
        
        subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)
print('number without t1 =',len(sub_no_t1))

Processing 796 subjects ...
52739556.dccn-l029.dccn.nl
52739557.dccn-l029.dccn.nl
52739558.dccn-l029.dccn.nl
52739559.dccn-l029.dccn.nl
52739560.dccn-l029.dccn.nl
52739561.dccn-l029.dccn.nl
52739562.dccn-l029.dccn.nl
52739563.dccn-l029.dccn.nl
52739564.dccn-l029.dccn.nl
52739565.dccn-l029.dccn.nl
52739566.dccn-l029.dccn.nl
52739567.dccn-l029.dccn.nl
52739568.dccn-l029.dccn.nl
52739569.dccn-l029.dccn.nl
52739570.dccn-l029.dccn.nl
52739571.dccn-l029.dccn.nl
52739572.dccn-l029.dccn.nl
52739573.dccn-l029.dccn.nl
52739574.dccn-l029.dccn.nl
52739575.dccn-l029.dccn.nl
52739576.dccn-l029.dccn.nl
52739577.dccn-l029.dccn.nl
52739578.dccn-l029.dccn.nl
52739579.dccn-l029.dccn.nl
52739580.dccn-l029.dccn.nl
52739581.dccn-l029.dccn.nl
52739582.dccn-l029.dccn.nl
52739583.dccn-l029.dccn.nl
52739584.dccn-l029.dccn.nl
52739585.dccn-l029.dccn.nl
52739586.dccn-l029.dccn.nl
52739587.dccn-l029.dccn.nl
52739588.dccn-l029.dccn.nl
52739589.dccn-l029.dccn.nl
52739590.dccn-l029.dccn.nl
52739591.dccn-l029.dccn.nl


qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of curren

jobs submitted = 2099
number of subjects already processed = 0
number failed = 0
number without t1 = 0


qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of current user's jobs exceeds the queue limit: user johbay@dccn-c035.dccn.nl, queue verylong)
qsub: submit error (Maximum number of jobs already in queue for user MSG=total number of curren

In [45]:
sessions

['/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0178_MR1',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0178_MR2',
 '/project_cephfs/3022017.06/OASIS2/freesurfer/OAS2_0178_MR3']